# AlexNet
- Minh Nguyen
- 12/15/2024
- Output (Height or Width) = (Input Size - Kernel Size + 2 x Padding) / Stride + 1
- About:
    - Designed for large-scaled ReLU activation and dropout for improved training.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        
        # feature extraction
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=11, stride=4, padding=2), # 3 x 224 x 224 -> 64 x 55 x 55
            nn.ReLU(inplace=True), 
            nn.MaxPool2d(kernel_size=3, stride=2), # 64 x 55 x 55 -> 64 x 27 x 27
            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, padding=2), # 64 x 27 x 27 -> 192 x 27 x 27
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2), # 192 x 27 x 27 -> 192 x 13 x 13
            nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, padding=1), # 192 x 13 x 13 -> 384 x 13 x 
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),          # Output: 256x13x13
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),          # Output: 256x13x13
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)                   # Output: 256x6x6
        )
        # classifier
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the feature map
        x = self.classifier(x)
        return x

class SmallAlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(SmallAlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1), # 1 x 28 x 28 -> 32 x 28 x 28
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),                 # 32 x 28 x 28 -> 32 x 14 x 14
            nn.Conv2d(32, 64, kernel_size=3, padding=1),           # 32 x 14 x 14 -> 64 x 14 x 14
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)                  # 64 x 14 x 14 -> 64 x 7 x 7
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * 7 * 7, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


# Prepare CIFAR-10
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize grayscale images
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define training settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = SmallAlexNet(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training Loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad() # reset the gradient
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward() # calculate the gradient
        optimizer.step() # update the parameters with the new gradient
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")    

# Testing Loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Using device: cpu
Epoch 1/10, Loss: 0.5321
Epoch 2/10, Loss: 0.3164
Epoch 3/10, Loss: 0.2663
Epoch 4/10, Loss: 0.2367
Epoch 5/10, Loss: 0.2133
Epoch 6/10, Loss: 0.1924
Epoch 7/10, Loss: 0.1751
Epoch 8/10, Loss: 0.1581
Epoch 9/10, Loss: 0.1428
Epoch 10/10, Loss: 0.1275
Test Accuracy: 92.13%
